In [76]:
import torch
import torch.nn as nn
from torch.nn import functional as F
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

block_size=8
batch_size=4
max_iters=10000
learning_rate=3e-4
eval_iters=250

tensor([1.], device='mps:0')


In [77]:
with open('wizard_of_oz_plaintext.txt', 'r', encoding='utf-8') as f:
    text=f.read()
chars=sorted(set(text))
print(chars)
vocab_size=len(chars)


['\n', ' ', '!', '#', '&', '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']


In [78]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate (chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])
data= torch.tensor(encode(text), dtype=torch.long)
#print(data[:200])

In [84]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(mps_device), y.to(mps_device)
    return x, y
    
x, y = get_batch('train')
print ('inputs: ')
# print (x.shape)
print(x)
print ('targets:')
print(y)
#print(x.device)


inputs: 
tensor([[60,  1, 54, 55, 65,  1, 48, 64],
        [57, 55, 60,  0, 69, 47, 65,  1],
        [51, 71, 65,  0, 54, 47, 50,  1],
        [61, 52,  1, 66, 54, 51,  1, 25]], device='mps:0')
targets:
tensor([[ 1, 54, 55, 65,  1, 48, 64, 51],
        [55, 60,  0, 69, 47, 65,  1, 61],
        [71, 65,  0, 54, 47, 50,  1, 54],
        [52,  1, 66, 54, 51,  1, 25, 61]], device='mps:0')


In [85]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [86]:

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print( 'when input is', context, 'target is', target)

when input is tensor([0]) target is tensor(0)
when input is tensor([0, 0]) target is tensor(38)
when input is tensor([ 0,  0, 38]) target is tensor(55)
when input is tensor([ 0,  0, 38, 55]) target is tensor(66)
when input is tensor([ 0,  0, 38, 55, 66]) target is tensor(58)
when input is tensor([ 0,  0, 38, 55, 66, 58]) target is tensor(51)
when input is tensor([ 0,  0, 38, 55, 66, 58, 51]) target is tensor(16)
when input is tensor([ 0,  0, 38, 55, 66, 58, 51, 16]) target is tensor(1)


In [87]:
class GPTLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size, vocab_size)
    def forward(self,index,targets=None):
        logits=self.token_embedding_table(index)
        if targets is None:
            loss=None
        else:
            B, T, C= logits.shape
            logits=logits.view(B*T, C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_tokens) :
        # index is (B, T) array of indices in the current context
        for _ in range (max_new_tokens) :
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial (probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch. cat ((index, index_next), dim=1) # (B, T+1)
        return index

model=BigramLanguageModel(vocab_size)
m=model.to(mps_device)

context=torch.zeros((1,1), dtype=torch.long, device=mps_device)
generated_chars=decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
    


g]w9F’ej CpPDCdz9(pAsDDKc&T;pSTL5H#3dZKuPcE[CuYNd?G9 fyjDDPe,enW2)Tn dT9Jur&OP?jbft:C Y,h(2).AzQxw’pN)U3wzyfWbXyeIqo”TXlyFpVqLF’’-O“-C!NcV3z”,“VPDkgxtvfC#e-N-U#3F’EgsK.)LAiRrE“.RS[D”fs2)rZ]V53bE’TXMLZdSvYXe[2dKi‘Exl’[j1)‘nI—I;r)lyo)&EXbOu“GnsGkZ2)URp-2q
w3S[5?Rp C,tEBZqX3mlKffSw’-dK(esP3o
aSpOHhO
YdSmg&]eeu“-BfS.dS&o5‘rEF’rEEZSLFgRQc30“vtW‘fB”sS[YXextPAJ[OFAOkj).Wu]pVKiOz
“gvDgRspJGrVf&Ku&z0ilFdJ!VZlUEaE3df!MDLYX
;wt]wz’LOoYbyPJ.KE’,)G23)s‘1?t)Ktq[c[X“hyo.“3FWP“Kt]Su‘ObXU.hh(cKRFcf#i‘Ti’e’1:k0“d


In [88]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.799, val loss: 4.777
step: 250, train loss: 4.755, val loss: 4.734
step: 500, train loss: 4.672, val loss: 4.684
step: 750, train loss: 4.590, val loss: 4.609
step: 1000, train loss: 4.528, val loss: 4.554
step: 1250, train loss: 4.492, val loss: 4.474
step: 1500, train loss: 4.435, val loss: 4.434
step: 1750, train loss: 4.360, val loss: 4.366
step: 2000, train loss: 4.305, val loss: 4.302
step: 2250, train loss: 4.266, val loss: 4.263
step: 2500, train loss: 4.210, val loss: 4.197
step: 2750, train loss: 4.149, val loss: 4.145
step: 3000, train loss: 4.103, val loss: 4.091
step: 3250, train loss: 4.047, val loss: 4.063
step: 3500, train loss: 4.012, val loss: 4.030
step: 3750, train loss: 3.957, val loss: 3.953
step: 4000, train loss: 3.899, val loss: 3.907
step: 4250, train loss: 3.846, val loss: 3.850
step: 4500, train loss: 3.817, val loss: 3.816
step: 4750, train loss: 3.775, val loss: 3.795
step: 5000, train loss: 3.710, val loss: 3.735
step: 5250, train l